# Prices

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [2]:
# Series stored as a dictionary
series = {'CUUR0000SA0': 'All items',
          'CUUR0000SAM': 'Medical care',
          'CUUR0000SAH': 'Housing',
          'CUUR0000SEFV': 'Food away from home',
          'CUUR0000SAF11': 'Food at home',
          'CUUR0000SAR': 'Recreation',
          'CUUR0000SAE1': 'Education',
          'CUUR0000SAT': 'Transportation',
          'CUUR0000SAA': 'Apparel',
          'CUUR0000SA0E': 'Energy',
          'CUUR0000SAE2': 'Communication'}

url = 'https://www.bls.gov/cpi/tables/relative-importance/2019.txt'
rel_imp = pd.read_csv(url, header=None, delim_whitespace=True)
r = requests.get(url)
data = r.text

wgts = {value: re.findall(value+r'\..*\s+(\d+\.\d+)\s+\d+\.\d+\s+\r', data)[0] 
        for key, value in series.items()}

In [3]:
# Start year and end year
years = (2018, 2020)
df = bls_api(series, years, bls_key)

Post Request Status: REQUEST_SUCCEEDED


In [4]:
wgt_date = pd.to_datetime('2019-12-01')
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}
    
results = pd.DataFrame(d).drop('All items').round(2).sort_values('Latest', ascending=False)
results.to_csv(data_dir / 'cpi_comp.csv', index_label='name')

mo1  = df.index[-1].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo1.txt', mo1)

mo2  = df.index[-13].strftime('%b %Y')
write_txt(text_dir / 'cpi_mo2.txt', mo2)

results

,Latest,Previous
Housing,1.02,1.22
Medical care,0.42,0.15
Food away from home,0.18,0.18
Food at home,0.08,0.11
Recreation,0.07,0.07
Education,0.06,0.10
Communication,0.04,-0.05
Apparel,-0.05,-0.07
Energy,-0.36,-0.03
Transportation,-0.37,0.08


In [5]:
ltdate = df.index[-1].strftime('%B %Y')
prevdate = df.index[-13].strftime('%B %Y')

ltd = {i: (results.iloc[i].name, results['Latest'].iloc[i]) for i in [0, 1, -2, -1]}

textd = {}
for idx, (name, val) in ltd.items():
    if idx == 0:
        name = name.lower()
    if val >= 0.05:
        textd[idx] = f'{name} contributed {val:.2f} percentage points to'
    elif val <= -0.04:
        textd[idx] = f'{name} subtracted {abs(val):.2f} percentage points from'
    else:
        textd[idx] = f'{name} did not contribute significantly to'
        
prevd = {i: (results.iloc[i].name, results['Previous'].iloc[i]) for i in [0, 1, -2, -1]}

textd2 = {}
for idx, (name, val) in prevd.items():
    if val >= 0.05:
        textd2[idx] = f'a contribution of {val:.2f} percentage points'
    elif val <= -0.04:
        textd2[idx] = f'a reduction of {abs(val):.2f} percentage points'
    else:
        textd2[idx] = 'virtually no effect on inflation'

In [6]:
text = (f'In {ltdate}, {textd[0]} overall CPI inflation, compared to '+
        f'{textd2[0]} in {prevdate}. {textd[1]} overall inflation in '+
        f'{ltdate}, compared to {textd2[1]} in {prevdate}.')

write_txt(text_dir / 'cpicomp.txt', text)

print(text)

text2 = (f'{textd[-1]} overall CPI inflation in {ltdate}, compared to '+
         f'{textd2[-1]} in {prevdate}. {textd[-2]} overall inflation, '+
         f'compared to {textd2[-2]} the previous year.')

write_txt(text_dir / 'cpicomp2.txt', text2)

text2

In March 2020, housing contributed 1.02 percentage points to overall CPI inflation, compared to a contribution of 1.22 percentage points in March 2019. Medical care contributed 0.42 percentage points to overall inflation in March 2020, compared to a contribution of 0.15 percentage points in March 2019.


'Transportation subtracted 0.37 percentage points from overall CPI inflation in March 2020, compared to a contribution of 0.08 percentage points in March 2019. Energy subtracted 0.36 percentage points from overall inflation, compared to virtually no effect on inflation the previous year.'

### CPI chart

In [7]:
# Series stored as a dictionary
series = {
    'CUSR0000SA0': 'ALL', 
    'CUSR0000SA0L1E': 'CORE'}

# Start year and end year
years = (1988, 2020)
df = bls_api(series, years, bls_key)

(df.pct_change(12).dropna() * 100).to_csv(data_dir / 'cpi.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### CPI text

In [8]:
s = pd.read_csv(data_dir / 'cpi.csv')
date = pd.to_datetime(s['date'].iloc[-1]).strftime('%B %Y')
core = s['CORE'].iloc[-1]
allitems = s['ALL'].iloc[-1]

text = ('\href{https://www.bls.gov/cpi/}{Consumer prices} increased by '+
        f'{allitems:.1f} percent over the year ending {date}, according '+
        'to the CPI for all urban consumers. Core inflation, which does '+
        'not include the more volatile food and energy prices, was '+
        f'{core:.1f} percent.')

write_txt(text_dir / 'cpi_main.txt', text)

text

'\\href{https://www.bls.gov/cpi/}{Consumer prices} increased by 1.5 percent over the year ending March 2020, according to the CPI for all urban consumers. Core inflation, which does not include the more volatile food and energy prices, was 2.1 percent.'

### PPI

In [2]:
ppi = (fred_df('PPIACO', start='1988')['VALUE'].pct_change(12) * 100).dropna()
ppi.to_csv(data_dir / 'ppi.csv', index_label='date')
node = end_node(ppi, 'green!80!blue')
write_txt(text_dir / 'ppi_node.txt', node)

ch = inc_dec_percent(ppi.iloc[-1])
prval = ppi.iloc[-13]
compare = compare_text(ppi.iloc[-1], prval, [1.0, 3.0, 5.0])
date = dtxt(ppi.index[-1])['mon1']
date2 = dtxt(ppi.index[-13])['mon1']

text = ('The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}{reports} '+
        'prices producers receive for the various goods and services they produce. '+
        'The producer price index for all commodities (see {\color{green!80!blue}\\textbf{---}}) '+
        f'{ch} over the year ending {date}, {compare} '+
        f'the 12-month growth rate of {prval:.1f} percent '+
        f'in {date2}.')

write_txt(text_dir / 'ppi_main.txt', text)

text

'The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}{reports} prices producers receive for the various goods and services they produce. The producer price index for all commodities (see {\\color{green!80!blue}\\textbf{---}}) decreased by 3.5 percent over the year ending March 2020, substantially below the 12-month growth rate of 0.8 percent in March 2019.'